In [63]:
import numpy as np
import pandas as pd

In [64]:
data = pd.read_csv('test01.pdb',sep='\s+')

data.columns = ['REMARK','4','1DPP','COMPLIES','X','Y','Z','2.0,','18-NOV-1998']
display(data)

,REMARK,4,1DPP,COMPLIES,X,Y,Z,"2.0,",18-NOV-1998
ATOM,1,C1,OLA,1,44.29,24.62,58.04,1.0,0.0
ATOM,2,C2,OLA,1,44.63,25.19,56.66,1.0,0.0
ATOM,3,C3,OLA,1,46.04,24.71,56.29,1.0,0.0
ATOM,4,C4,OLA,1,46.63,25.14,54.94,1.0,0.0
ATOM,5,C5,OLA,1,48.05,24.57,54.94,1.0,0.0
ATOM,6,C6,OLA,1,48.83,25.06,53.72,1.0,0.0
ATOM,7,C7,OLA,1,48.14,24.56,52.45,1.0,0.0
ATOM,8,C8,OLA,1,48.63,24.69,51.00,1.0,0.0
ATOM,9,C9,OLA,1,47.74,24.05,49.94,1.0,0.0
ATOM,10,C10,OLA,1,47.84,24.38,48.59,1.0,0.0


In [65]:
def get_angle(a, b, c):
    ba = a - b
    bc = c - b
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(cosine_angle)
    return(np.degrees(angle))

In [66]:
data.index = range(len(data))
a = data.loc[0, ['X', 'Y', 'Z']]
b = data.loc[1, ['X', 'Y', 'Z']]
c = data.loc[2, ['X', 'Y', 'Z']]

In [67]:
get_angle(a, b, c)

107.74807899719426

In [68]:
def dots_angle(data):
    data.index = range(len(data))
    angles = [0]
    for k in range(len(data)-2):
        a = data.loc[k, ['X', 'Y', 'Z']]
        b = data.loc[k+1, ['X', 'Y', 'Z']]
        c = data.loc[k+2, ['X', 'Y', 'Z']]
        angles.append(get_angle(a, b, c))
    angles.append(0)
    return(angles)

In [69]:
data['dots-angle'] = dots_angle(data)

In [70]:
display(data)

,REMARK,4,1DPP,COMPLIES,X,Y,Z,"2.0,",18-NOV-1998,dots-angle
0,1,C1,OLA,1,44.29,24.62,58.04,1.0,0.0,0.000000
1,2,C2,OLA,1,44.63,25.19,56.66,1.0,0.0,107.748079
2,3,C3,OLA,1,46.04,24.71,56.29,1.0,0.0,118.529798
3,4,C4,OLA,1,46.63,25.14,54.94,1.0,0.0,104.618734
4,5,C5,OLA,1,48.05,24.57,54.94,1.0,0.0,110.738952
5,6,C6,OLA,1,48.83,25.06,53.72,1.0,0.0,109.130631
6,7,C7,OLA,1,48.14,24.56,52.45,1.0,0.0,127.755005
7,8,C8,OLA,1,48.63,24.69,51.00,1.0,0.0,115.752242
8,9,C9,OLA,1,47.74,24.05,49.94,1.0,0.0,122.155375
9,10,C10,OLA,1,47.84,24.38,48.59,1.0,0.0,126.320389
